<a href="https://colab.research.google.com/github/AjManoj/Experiment/blob/master/NLP_Browser_History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyppeteer

# Imports

In [3]:
import requests
import asyncio
from pyppeteer import launch

from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import json
import pandas as pd
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def fetchData(url):
  try:
    page = requests.get(url)
    return text_from_html(page.content)
     
  except:
    return ''

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving BrowserHistory.json to BrowserHistory (1).json
User uploaded file "BrowserHistory.json" with length 36166727 bytes


# Input Data

In [4]:
#f=uploaded['BrowserHistory.json']
f=open('BrowserHistory.json', "r", encoding='utf-8')
data = json.load(f)
df = pd.json_normalize(data['Browser History'])

In [5]:
df.head(2)

,favicon_url,page_transition,title,url,client_id,time_usec
0,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/,CaUzdWXFXPEbl88cMHSoeQ==,1650022343257474
1,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/,CaUzdWXFXPEbl88cMHSoeQ==,1650022334393672


In [6]:
df.shape

(82904, 6)

In [7]:
df=df.drop_duplicates(['url','title'], keep='first')

In [8]:
df.shape

(36537, 6)

In [9]:
limitedDf = df.copy()#[0:100]

In [ ]:
limitedDf['Content'] = limitedDf['url'].progress_apply(lambda url:fetchData(url))

  0%|          | 0/36537 [00:00<?, ?it/s]

In [ ]:
file_name ='/content/drive/MyDrive/browserHistory.csv'
limitedDf.to_csv(file_name, encoding='utf-8', index=False)
